## 0. Dependencies

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import time
import pandas as pd
import csv
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , precision_score , recall_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression , RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pickle

In [2]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

mediapipe==0.9.0
cv2==4.6.0
numpy==1.21.5
pandas==1.4.2
csv==1.0


In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

## 1. Detections

In [14]:
cap = cv2.VideoCapture('v1.mp4',apiPreference=cv2.CAP_MSMF)

# model initiation
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        
        # Feed color conversion
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # pose detections
        results = pose.process(image)
        
        # revert back to bgr for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                  ,mp_drawing.DrawingSpec(color=(245,117,66),
                                                          thickness=2, circle_radius=4)
                                  ,mp_drawing.DrawingSpec(color=(245,66,230),
                                                          thickness=2,circle_radius=2))
        cv2.imshow('Raw Feed',image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()
    

## 1.1 Video for Training

In [13]:
cap =cv2.VideoCapture('v4.mp4')

height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
vwriter = cv2.VideoWriter('ded1.mp4',cv2.VideoWriter_fourcc('P','I','M','1'), 
                          fps, (int(width), int(height)))
time.sleep(5)
while cap.isOpened():
    ret,frame = cap.read()
    
    try: 
        cv2.imshow("Training Video",frame)
        vwriter.write(frame)
    except Exception as e:
        break
        
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
vwriter.release()
cv2.destroyAllWindows()

## 2. Extracting coordinates

In [6]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [8]:
landmarks = ['class']
for val in range(1,34):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [7]:
landmarks

NameError: name 'landmarks' is not defined

In [8]:
with open("coords.csv",mode='w',newline='') as f:
    csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [8]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x,res.y,
                        res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)
        
        with open('coords.csv',mode='a',newline='') as f:
            csv_writer = csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)

In [9]:
export_landmark(results,'up')

In [12]:
results.pose_landmarks.landmark

[x: 0.4719256
y: 0.051216282
z: 0.26841727
visibility: 0.99943215
, x: 0.4694388
y: 0.04037122
z: 0.24188228
visibility: 0.99932116
, x: 0.46713042
y: 0.03904644
z: 0.24187854
visibility: 0.99935937
, x: 0.4650374
y: 0.037516646
z: 0.24184926
visibility: 0.9992734
, x: 0.47691065
y: 0.04275387
z: 0.24318056
visibility: 0.99929017
, x: 0.47948736
y: 0.042950306
z: 0.24322768
visibility: 0.99930376
, x: 0.4823617
y: 0.04265015
z: 0.2431539
visibility: 0.9992277
, x: 0.46064982
y: 0.038756765
z: 0.14295341
visibility: 0.9995671
, x: 0.48478436
y: 0.04461633
z: 0.14826149
visibility: 0.9993391
, x: 0.46831074
y: 0.05909084
z: 0.23851559
visibility: 0.99938154
, x: 0.4753035
y: 0.060016777
z: 0.23995042
visibility: 0.9992825
, x: 0.43164387
y: 0.08094204
z: 0.0773942
visibility: 0.99992627
, x: 0.50681007
y: 0.08634732
z: 0.08848335
visibility: 0.9998875
, x: 0.41165978
y: 0.18946266
z: 0.08470501
visibility: 0.9450279
, x: 0.5294405
y: 0.20453115
z: 0.0868011
visibility: 0.92577225
, x: 0.

In [27]:
cap = cv2.VideoCapture('v5.mp4',apiPreference=cv2.CAP_MSMF)
#Initiate model
with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
        #recolor
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Make Detections
        results = pose.process(image)
        
        #Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                                  ,mp_drawing.DrawingSpec(color=(245,117,66),
                                                          thickness=2, circle_radius=4)
                                  ,mp_drawing.DrawingSpec(color=(245,66,230),
                                                          thickness=2,circle_radius=2))
        k = cv2.waitKey(1)
        if k == 117:
            export_landmark(results,'up')
        if k == 100:
            export_landmark(results,'down')
        
        cv2.imshow('Raw Webcam Feed',image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break;
cap.release()
cv2.destroyAllWindows()

## 3. Training 

In [48]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [49]:
df = pd.read_csv('coords.csv')

In [50]:
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,up,0.553414,0.534331,-0.857037,0.999868,0.578112,0.459495,-0.823187,0.999740,0.594897,...,0.278216,0.000019,0.582104,2.896028,-0.188118,0.000063,0.409083,2.891962,-0.296995,0.000027
1,up,0.298316,0.217357,-0.333526,0.999911,0.307454,0.203464,-0.314020,0.999759,0.312366,...,0.134620,0.305781,0.385195,0.937053,0.014909,0.450979,0.270425,0.942263,-0.036056,0.497464
2,down,0.365290,0.439646,-0.628573,0.998388,0.375520,0.415676,-0.624889,0.996742,0.380499,...,-0.030983,0.108412,0.445133,0.867158,-0.119507,0.190882,0.336602,0.883210,-0.153117,0.176909
3,up,0.373031,0.186182,-0.370811,0.999881,0.376678,0.173634,-0.349738,0.999689,0.380530,...,0.208197,0.398352,0.423046,0.905484,0.164315,0.514231,0.340754,0.935973,0.039783,0.591700
4,up,0.370073,0.179774,-0.381279,0.999907,0.373892,0.169214,-0.359603,0.999739,0.377747,...,0.219616,0.381198,0.422675,0.910563,0.145387,0.493635,0.344373,0.935439,0.055919,0.569865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,down,0.505912,0.342064,-0.571615,0.999977,0.509332,0.321204,-0.571421,0.999955,0.512551,...,0.012669,0.657344,0.597133,0.765527,-0.058824,0.737504,0.402833,0.790975,-0.090179,0.792935
1616,down,0.510507,0.357249,-0.569984,0.999971,0.514529,0.339799,-0.566404,0.999944,0.517616,...,0.004810,0.613308,0.582749,0.768927,-0.094313,0.688998,0.402705,0.790554,-0.100184,0.749907
1617,down,0.513802,0.361378,-0.480517,0.999970,0.518541,0.344641,-0.480635,0.999947,0.521607,...,0.004908,0.593730,0.571724,0.755795,-0.062082,0.660315,0.402678,0.789983,-0.098454,0.739275
1618,down,0.515294,0.355196,-0.475967,0.999962,0.520098,0.338118,-0.475914,0.999940,0.523091,...,0.026334,0.570312,0.577899,0.758054,-0.021524,0.624968,0.403091,0.786997,-0.071798,0.724207


In [51]:
x = df.drop('class',axis = 1)
y = df['class']

In [52]:
x

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,0.553414,0.534331,-0.857037,0.999868,0.578112,0.459495,-0.823187,0.999740,0.594897,0.458089,...,0.278216,0.000019,0.582104,2.896028,-0.188118,0.000063,0.409083,2.891962,-0.296995,0.000027
1,0.298316,0.217357,-0.333526,0.999911,0.307454,0.203464,-0.314020,0.999759,0.312366,0.203748,...,0.134620,0.305781,0.385195,0.937053,0.014909,0.450979,0.270425,0.942263,-0.036056,0.497464
2,0.365290,0.439646,-0.628573,0.998388,0.375520,0.415676,-0.624889,0.996742,0.380499,0.413431,...,-0.030983,0.108412,0.445133,0.867158,-0.119507,0.190882,0.336602,0.883210,-0.153117,0.176909
3,0.373031,0.186182,-0.370811,0.999881,0.376678,0.173634,-0.349738,0.999689,0.380530,0.173480,...,0.208197,0.398352,0.423046,0.905484,0.164315,0.514231,0.340754,0.935973,0.039783,0.591700
4,0.370073,0.179774,-0.381279,0.999907,0.373892,0.169214,-0.359603,0.999739,0.377747,0.169542,...,0.219616,0.381198,0.422675,0.910563,0.145387,0.493635,0.344373,0.935439,0.055919,0.569865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1615,0.505912,0.342064,-0.571615,0.999977,0.509332,0.321204,-0.571421,0.999955,0.512551,0.317854,...,0.012669,0.657344,0.597133,0.765527,-0.058824,0.737504,0.402833,0.790975,-0.090179,0.792935
1616,0.510507,0.357249,-0.569984,0.999971,0.514529,0.339799,-0.566404,0.999944,0.517616,0.336640,...,0.004810,0.613308,0.582749,0.768927,-0.094313,0.688998,0.402705,0.790554,-0.100184,0.749907
1617,0.513802,0.361378,-0.480517,0.999970,0.518541,0.344641,-0.480635,0.999947,0.521607,0.341343,...,0.004908,0.593730,0.571724,0.755795,-0.062082,0.660315,0.402678,0.789983,-0.098454,0.739275
1618,0.515294,0.355196,-0.475967,0.999962,0.520098,0.338118,-0.475914,0.999940,0.523091,0.335115,...,0.026334,0.570312,0.577899,0.758054,-0.021524,0.624968,0.403091,0.786997,-0.071798,0.724207


In [53]:
y

0         up
1         up
2       down
3         up
4         up
        ... 
1615    down
1616    down
1617    down
1618    down
1619    down
Name: class, Length: 1620, dtype: object

In [54]:
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3,random_state = 1234)

In [55]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression , RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [56]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [57]:
fit_model = {}
for algo , pipeline in pipelines.items():
    model = pipeline.fit(x_train,y_train)
    fit_model[algo] = model


In [58]:
fit_model['rc'].predict(x_test)

array(['down', 'up', 'down', 'up', 'down', 'down', 'down', 'down', 'up',
       'up', 'down', 'down', 'up', 'down', 'down', 'up', 'down', 'down',
       'up', 'down', 'up', 'down', 'down', 'up', 'up', 'down', 'up',
       'down', 'down', 'down', 'down', 'down', 'up', 'up', 'up', 'down',
       'down', 'down', 'down', 'up', 'up', 'up', 'up', 'up', 'up', 'down',
       'up', 'down', 'up', 'up', 'down', 'down', 'down', 'down', 'down',
       'up', 'down', 'up', 'down', 'up', 'down', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'down', 'up',
       'down', 'down', 'up', 'down', 'down', 'up', 'down', 'up', 'down',
       'up', 'up', 'down', 'up', 'up', 'down', 'up', 'down', 'up', 'up',
       'down', 'up', 'down', 'up', 'down', 'up', 'up', 'down', 'down',
       'up', 'down', 'up', 'down', 'up', 'down', 'up', 'up', 'up', 'down',
       'up', 'down', 'down', 'up', 'down', 'down', 'down', 'up', 'down',
       'down', 'up', 'up', 'down', 'down', 'up', 'down', 

In [41]:
y_test.head(10)

1595    down
1147      up
414     down
1251      up
218     down
1568    down
831     down
1086    down
972       up
643       up
Name: class, dtype: object

## 4. Evaluation

In [62]:
from sklearn.metrics import accuracy_score , precision_score , recall_score
import pickle

In [63]:
for algo,model in fit_model.items():
    yhat = model.predict(x_test)
    print(algo,accuracy_score(y_test.values,yhat),
         precision_score(y_test.values,yhat,average = "binary",pos_label = "up"),
         recall_score(y_test.values,yhat,average = "binary",pos_label = "up"))

lr 0.9917695473251029 0.9884615384615385 0.9961240310077519
rc 0.9917695473251029 0.9847328244274809 1.0
rf 0.9917695473251029 0.9884615384615385 0.9961240310077519
gb 0.9876543209876543 0.9846153846153847 0.9922480620155039


In [34]:
yhat = fit_model['rf'].predict(x_test)
yhat[:10]

array(['down', 'down', 'up', 'down', 'down', 'down', 'up', 'up', 'up',
       'up'], dtype=object)

## 5. Save/Load Model using pickle

In [67]:
with open('model.pkl','wb') as f:
    pickle.dump(fit_model['rf'],f)

In [68]:
with open('model.pkl','rb') as f:
    model = pickle.load(f)

In [69]:
model.predict(x_test)

array(['down', 'up', 'down', 'up', 'down', 'down', 'down', 'down', 'up',
       'up', 'down', 'down', 'up', 'down', 'down', 'up', 'down', 'down',
       'up', 'down', 'up', 'down', 'down', 'up', 'up', 'down', 'up',
       'down', 'down', 'down', 'down', 'down', 'up', 'up', 'up', 'down',
       'down', 'down', 'down', 'up', 'up', 'up', 'up', 'up', 'up', 'down',
       'up', 'down', 'up', 'up', 'down', 'down', 'down', 'down', 'down',
       'up', 'down', 'up', 'down', 'up', 'down', 'up', 'up', 'up', 'up',
       'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'down', 'up',
       'down', 'down', 'up', 'down', 'down', 'up', 'down', 'up', 'down',
       'up', 'up', 'down', 'up', 'up', 'down', 'up', 'down', 'up', 'up',
       'down', 'up', 'down', 'up', 'down', 'up', 'up', 'down', 'down',
       'up', 'down', 'up', 'down', 'up', 'down', 'up', 'up', 'up', 'down',
       'up', 'down', 'down', 'up', 'down', 'down', 'down', 'up', 'down',
       'down', 'up', 'up', 'down', 'down', 'up', 'down', 